## Build a RAG 

Dieses Skript implementiert ein Retrieval-Augented-Generation System mit LangSmith, um die Leistung zu verfolgen und Fehler zu analysieren.
Außerdem wurde die Funktion des dynamischen Änderens von Prompts implementiert, um die Antwortqualität zu optimieren.

In [3]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langsmith.run_helpers import traceable   
import os

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\hshakademie3\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: '_Environ' object is not callable

In [ ]:
# 🔐 



# تعيين المتغيرات لمكتبة LangSmith
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
os.environ["HF_TOKEN"] = "HF_TOKEN"

In [16]:

url = "https://www.spiegel.de/wirtschaft/unternehmen/tesla-entlaesst-ueber-10000-mitarbeiter-wegen-absatzproblemen-a-xyz"
loader = WebBaseLoader(url)
document = loader.load()[0].page_content.strip().replace("\n", " ")


In [17]:
# تقسيم النص إلى مقاطع صغيرة sinnvoll chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.create_documents([document])

In [18]:
persist_directory = "chroma_db_rag"
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


embedding.embed_query('hi')

db = Chroma.from_documents(documents=docs,
                           embedding=embedding,
                           persist_directory=persist_directory)
retriever = db.as_retriever()
db.persist()


C:\Users\hshakademie3\AppData\Local\Temp\ipykernel_19216\325043896.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\hshakademie3\AppData\Local\Temp\ipykernel_19216\325043896.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [19]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


C:\Users\hshakademie3\AppData\Local\Temp\ipykernel_19216\1046491335.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [21]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)
rag_chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True
)

In [22]:
def filter_documents_by_metadata(metadata_key, metadata_value):
    # فلترة المستندات بناءً على الميتاداتا
    filtered_docs = []
    for doc in docs:
        if doc.metadata.get(metadata_key) == metadata_value:
            filtered_docs.append(doc)
    return filtered_docs


In [23]:
def multi_query_retrieval(queries):
    # استرجاع الوثائق باستخدام استعلامات متعددة
    answers = []
    for query in queries:
        result = rag_chain.invoke({"question": query})
        answers.append(result['answer'])
    return answers

In [24]:
@traceable
def answer_question(frage):
    return rag_chain.invoke({"question": frage})

In [25]:
fragen = [
    "Warum hat Tesla im Jahr 2024 so viele Mitarbeiter entlassen?",
    "Was ist der Grund für den Umsatzrückgang von Tesla?",
    "Wann begann der Stellenabbau bei Tesla?",
    "Warum hat Tesla Mitarbeiter entlassen? Was waren die Ursachen?",
    "Welches Modell von Tesla hatte den größten Rückgang in den Verkaufszahlen?"
]

In [26]:
# Variieren Sie die Prompts und analysieren Sie deren Auswirkungen
prompts = [
    "Was waren die Gründe für die Entlassungen bei Tesla im Jahr 2024?",
    "Warum hat Tesla viele Mitarbeiter in 2024 entlassen? Erklären Sie den Zusammenhang.",
    "Gibt es spezifische Ursachen für den Rückgang des Tesla-Umsatzes 2024?",
    "Welche Faktoren haben zu den Entlassungen bei Tesla geführt?",
    "Warum ist der Umsatz von Tesla im Jahr 2024 gefallen?"
]


In [8]:
import os
print("🔑 Google API Key:", os.environ["GOOGLE_API_KEY"])


🔑 Google API Key: GOOGLE_API_KEY


In [2]:
multi_query_results = multi_query_retrieval(prompts)
print("\n🔄 Ergebnisse der Multi-Query Retrieval:")
for result in multi_query_results:
    print("\n💡 Antwort:", result)

NameError: name 'multi_query_retrieval' is not defined

In [ ]:
for frage in fragen:
    print("\n❓ Frage:", frage)
    result = answer_question(frage)
    print("\n💡 Antwort:", result['answer'])
    print("📚 Quelle:", result['source_documents'][0].metadata.get('source', 'Unbekannt'))


# 📦 1. تحميل المكتبات
# -----------------------------
# 📰 3. تحميل ومعالجة الوثيقة
# -----------------------------
# تحميل مقال حديث بعد أغسطس 2024
# تقسيم النص إلى مقاطع صغيرة sinnvoll chunks
# -----------------------------
# 💾 4. إنشاء قاعدة بيانات向量ية - Chroma
# -----------------------------
# 🧠 5. إعداد الذاكرة للحوار
# -----------------------------
# 🤖 6. إعداد النموذج والـ RAG Chain
# -----------------------------
# 🔍 7. فلترة الميتاداتا - Bonus
# -----------------------------
# 🔄 8. Multi-Query Retrieval - Bonus
# -----------------------------
# 💬 9. دالة للإجابة على الأسئلة مع LangSmith
# -----------------------------
# 🧪 10. اختبار النظام - Beispiele
# -----------------------------
# اختبار مع تغيير الـ Prompts
# إجراء استعلامات متعددة
# اختبار النظام باستخدام استعلام واحد